In [1]:
## Import Required Libraries

In [2]:
import os  # Importing the os module to interact with the operating system for environment variable management.
import pandas as pd  # Importing pandas for data manipulation and analysis, particularly for handling data frames.
import pyodbc  # Importing pyodbc to facilitate connections to SQL databases using ODBC.
import nltk  # Importing the Natural Language Toolkit (NLTK) for natural language processing tasks.
from nltk.sentiment.vader import SentimentIntensityAnalyzer  # Importing the VADER sentiment analysis tool for sentiment scoring.

import warnings  # Importing the warnings module to manage warning messages in the output.
warnings.filterwarnings('ignore')  # Suppressing warnings to maintain a cleaner output during execution.

In [16]:
nltk.download('vader_lexicon') # Download the VADER lexicon for sentiment analysis.

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...


True

In [3]:
server = os.getenv('server')  # Retrieve the SQL Server instance name from the environment variable.
database = os.getenv('database')  # Retrieve the database name from the environment variable.

In [16]:
# Function to retrieve customer reviews from SQL Server
def get_customer_reviews():
    # Connection string for SQL Server
    conn_str = (
        "Driver={ODBC Driver 17 for SQL Server};"  # Specify the ODBC driver
        f"Server={server};"                         # Server instance name
        f"Database={database};"                     # Database name
        "Trusted_Connection=yes;"                   # Use trusted connection
    )
    
    conn = pyodbc.connect(conn_str)  # Establish the connection to the database
    
    # SQL query to retrieve cleaned customer reviews
    sql_query = "SELECT review_id, customer_id, product_id, review_date, rating, review_text FROM cleaned_customer_reviews"
    
    df = pd.read_sql(sql_query, conn)  # Execute the query and store the results in a DataFrame
    
    conn.close()  # Close the database connection
    
    return df  # Return the DataFrame containing the fetched data

In [17]:
# Fetch customer reviews into a DataFrame
customer_reviews_df = get_customer_reviews()  # Call the function to retrieve customer reviews

In [18]:
# Initialize the Sentiment Intensity Analyzer
analyzer = SentimentIntensityAnalyzer()  # Create an instance of the VADER sentiment analyzer

In [19]:
# Calculate the sentiment score of a review
def get_sentiment_score(review):
    sentiment = analyzer.polarity_scores(review)  # Get sentiment scores for the review
    return sentiment['compound']  # Return the compound sentiment score

In [20]:
# Categorize sentiment based on score and rating
def determine_sentiment_category(score, rating):
    if score > 0.05:
        if rating > 3:
            return 'Positive'
        elif rating == 3:
            return 'Mixed Positive'
        else:
            return 'Mixed Negative'
    elif score < -0.05:
        if rating < 3:
            return 'Negative'
        elif rating == 3:
            return 'Mixed Negative'
        else:
            return 'Mixed Positive'
    else:
        if rating > 3:
            return 'Positive'
        elif rating < 3:
            return 'Negative'
        else:
            return 'Neutral'

In [21]:
# Assign sentiment score to a defined bucket
def classify_sentiment_bucket(score):
    if score >= 0.5:
        return '0.5 to 1.0'
    elif score >= 0.0 and score < 0.5:
        return '0.0 to 0.49'
    elif score <= -0.5:
        return '-1.0 to -0.5'
    else:
        return '-0.49 to 0.0'

In [22]:
customer_reviews_df['sentiment_score']=customer_reviews_df['review_text'].apply(get_sentiment_score)

In [23]:
customer_reviews_df['sentiment_bucket'] = customer_reviews_df['sentiment_score'].apply(classify_sentiment_bucket)

In [24]:
customer_reviews_df['sentiment_category']=customer_reviews_df.apply(lambda row: determine_sentiment_category(row['sentiment_score'],row['rating']),axis=1)

In [25]:
customer_reviews_df.head()

,review_id,customer_id,product_id,review_date,rating,review_text,sentiment_score,sentiment_bucket,sentiment_category
0,1,77,18,2021-12-23,3,"average experience, nothing special.",-0.3089,-0.49 to 0.0,Mixed Negative
1,2,80,19,2022-12-25,5,the quality is top-notch.,0.0000,0.0 to 0.49,Positive
2,3,50,13,2023-01-26,4,five stars for the quick delivery.,0.0000,0.0 to 0.49,Positive
3,4,78,15,2023-04-21,3,"good quality, but could be cheaper.",0.2382,0.0 to 0.49,Mixed Positive
4,5,64,2,2021-07-16,3,"average experience, nothing special.",-0.3089,-0.49 to 0.0,Mixed Negative


In [26]:
customer_reviews_df.to_csv('customer_reviews_with_sentiments.csv',index=False)